## KubeVirt

Dieses Notebook enthält die notwendigen Befehle zur Installation, Konfiguration und Nutzung von KubeVirt für virtuelle Maschinen auf Kubernetes.

**Installation von KubeVirt**

Dazu greifen wir auf Scripts aus dem [Lern Cloud Projekt](https://github.com/mc-b/lerncloud/tree/main/services) zurück


In [ ]:
%%bash
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/kubevirt.sh | bash -

## Testen

### Variante A: VM ab Container-Image

Das OS-Image wurde von den kubevirt Personen als Container Image verpackt.

Ist einfacher zu starten, bedeutet aber auch das zusätzliche Arbeit nötig.

In [ ]:
%%bash
kubectl apply -f https://kubevirt.io/labs/manifests/vm.yaml

In [ ]:
%%bash
virtctl start testvm

Kontrolle ob Storage initialisert wurde und VM startet

In [ ]:
%%bash
kubectl get sc,pv,pvc,vm,vmi

Verbinden mittels der Console, muss in der VM erfolgen:

    virtctl console testvm
    
exit mittels `ctrl+]`    

Wenn es nicht läuft, Logs vom virt-handler im Namespace `kubevirt` prüfen

In [ ]:
%%bash
kubectl logs -n kubevirt daemonset/virt-handler

### Variante B: VM mittels Containerized Data Importer

Das OS-Image (normaleweise mit Cloud-Image bezeichnet) kann dabei 1:1 von der entsprechenden Downloadsite verwendet werden. 

Ein Verpacken als Container Image entfällt.

Dies braucht jedoch eine weitere Kubernetes Erweiterung den **Containerized Data Importer**.

### Erstellen des Data Volumes für Debian Cloud-Image

Mit dieser Variante wird das OS-Image auf einem **Persistent Volume Claim** abgelegt.

Für diese Variante braucht es einen installierten StorageClass Driver (z.B. [microk8 hostpath-storage](https://microk8s.io/docs/addon-hostpath-storage), [longhorn](https://longhorn.io/), etc.)

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cdi.kubevirt.io/v1beta1
kind: DataVolume
metadata:
  name: "debian"
spec:
  source:
    http:
      url: "https://cloud.debian.org/images/cloud/bullseye/latest/debian-11-generic-amd64.qcow2"
  pvc:
    accessModes:
    - ReadWriteOnce
    resources:
      requests:
        storage: "4Gi"         
EOF


### Erstellen der Debian VM

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: kubevirt.io/v1
kind: VirtualMachine
metadata:
  labels:
    kubevirt.io/os: linux
  name: debian
spec:
  running: true
  template:
    metadata:
      creationTimestamp: null
      labels:
        kubevirt.io/domain: debian
    spec:
      domain:
        cpu:
          cores: 1
        devices:
          disks:
          - disk:
              bus: virtio
            name: disk0
          - cdrom:
              bus: sata
              readonly: true
            name: cloudinitdisk
        resources:
          requests:
            memory: 2G
      volumes:
      - name: disk0
        persistentVolumeClaim:
          claimName: debian
      - cloudInitNoCloud:
          userData: |
            #cloud-config
            system_info:
              default_user:
                name: debian
                home: /home/debian
            password: insecure
            chpasswd: { expire: False }
            hostname: debian-k8s
            ssh_pwauth: True
            disable_root: false
            ssh_authorized_keys:
            - ssh-rsa AAAAB3Nza...YOUR_SSH_KEY...insecure@lerncloud
            packages:
              - qemu-guest-agent               
        name: cloudinitdisk
EOF


### Kontrolle der Einrichtung

In [ ]:
%%bash
kubectl get sc,pv,pvc,dv,vm,vmi -o wide

- - -

## Aufträge

* Erweitert das Cloud-init Script von der Debian VM um die Installation von `nginx`.
* Erstellt einen Service um auf den `nginx` welcher in der VM läuft zugreifen zu können.
* Verwendet Ubuntu statt Debian als OS für die VM.
* Deaktiviert die Emulation (VM in VM) und versucht die VM direkt auf der K8s Installation laufen zu lassen.

### Links
- [KubeVirt User Guide](https://kubevirt.io/user-guide/cluster_admin/installation/)
- [Run VMs like a Pod](https://eng.d2iq.com/blog/kubevirt-part-1-run-vms-like-a-pod/)
- [Ubuntu Installations Images](https://cloud-images.ubuntu.com/)

- - -
## Aufräumen von KubeVirt und CDI

In [ ]:
%%bash
kubectl delete vm debian
kubectl delete datavolume debian

export VERSION=$(curl -Ls https://github.com/kubevirt/containerized-data-importer/releases/latest | grep -m 1 -o "v[0-9]\.[0-9]*\.[0-9]*")
kubectl delete -f https://github.com/kubevirt/containerized-data-importer/releases/download/$VERSION/cdi-operator.yaml
kubectl delete -f https://github.com/kubevirt/containerized-data-importer/releases/download/$VERSION/cdi-cr.yaml    

export VERSION=$(curl https://storage.googleapis.com/kubevirt-prow/release/kubevirt/kubevirt/stable.txt)
kubectl delete -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-operator.yaml    
kubectl delete -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-cr.yaml    
